<h1> Extract, Load and Transform (ETL) process

Here we import each one of the datasets and transform them in order to export them and load into the dashboard.

In [1]:
#Import libraries:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from google.colab import drive

In [2]:
# Mount on Google Drive
drive.mount('/content/drive/')

Mounted at /content/drive/


<h2> Product dataframe </h2>

In [3]:
product_df_raw = pd.read_csv('/content/drive/MyDrive/Analytics/Datasets/product_sales_sample.csv')
product_df_raw

,Rest Owner,Rest Coop,REST_KEY,MITM_KEY,Menu Item,Menu Item Combo Meal Flag,Reporting Day,POS Consumer Price,POS Total Units Sold Promo and Regular,POS Promotion Units Sold,POS Units Sold,POS Combo Units Sold
0,SMITH JOE,SEA/TCA WA CP-0024,1364,1,00000001-REUBEN,N,1/1/11,0.95,25,0,25,6
1,SMITH JOE,SEA/TCA WA CP-0024,5357,1,00000001-REUBEN,N,1/1/11,0.95,10,0,10,1
2,SMITH JOE,SEA/TCA WA CP-0024,13369,1,00000001-REUBEN,N,1/1/11,0.95,9,0,9,3
3,SMITH JOE,SEA/TCA WA CP-0024,13604,1,00000001-REUBEN,N,1/1/11,0.95,24,0,24,4
4,SMITH JOE,SEA/TCA WA CP-0024,1364,1,00000001-REUBEN,N,1/2/11,0.95,28,0,28,4
...,...,...,...,...,...,...,...,...,...,...,...,...
6511,SMITH JOE,SEA/TCA WA CP-0024,13369,134,00000134-SML ICED CRML MOCHA,N,1/24/11,2.35,2,0,2,0
6512,SMITH JOE,SEA/TCA WA CP-0024,13369,134,00000134-SML ICED CRML MOCHA,N,1/25/11,2.35,1,0,1,0
6513,SMITH JOE,SEA/TCA WA CP-0024,5357,134,00000134-SML ICED CRML MOCHA,N,1/27/11,2.35,1,0,1,0
6514,SMITH JOE,SEA/TCA WA CP-0024,13369,134,00000134-SML ICED CRML MOCHA,N,1/29/11,2.35,2,0,2,0


In [4]:
product_df_raw.columns = [x.strip() for x in product_df_raw.columns]
product_df_raw.columns = [x.replace('  ', ' ') for x in product_df_raw.columns]

In [5]:
product_df_raw['Reporting Day'] = pd.to_datetime(product_df_raw['Reporting Day'])
product_df_raw['day'] = product_df_raw['Reporting Day'].dt.day

<ipython-input-5-28b101f01c75>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  product_df_raw['Reporting Day'] = pd.to_datetime(product_df_raw['Reporting Day'])


We create the grouped dataframe at day level to plot the time series in the period of interest.

In [6]:
#Grouping and summing the 'POS Total Units Sold Promo and Regular' per day:
grouped_df_pos_total = product_df_raw[['day', 'POS Total Units Sold Promo and Regular']].groupby('day').sum()

In [7]:
grouped_df_pos_total

,POS Total Units Sold Promo and Regular
day,
1,4952
2,5305
3,6144
4,6018
5,6273
6,6911
7,6871
8,5417
9,5282


In [8]:
grouped_df_pos_total.to_csv('/content/drive/MyDrive/Analytics/Dashboard/sales_df.csv', sep=';')

We create the grouped dataframe at day level and store to plot the time series in the period of interest grouped by store.

In [9]:
mapping_dict = {
    1364: 'Store1',
    5357: 'Store2',
    13369: 'Store3',
    13604: 'Store4'
}

# Use the dictionary to transform the REST_KEY column
encoded_rest_key = product_df_raw['REST_KEY'].map(mapping_dict)

product_df_raw['Encoded_REST_KEY'] = encoded_rest_key

In [10]:
grouped_df_pd = product_df_raw[['Encoded_REST_KEY', 'day',
                                'POS Total Units Sold Promo and Regular']].groupby(['Encoded_REST_KEY','day']).sum()

In [11]:
grouped_df_pd.to_csv('/content/drive/MyDrive/Analytics/Dashboard/sales_store_df.csv', sep=';')

We create the grouped dataframe at day of the week level to plot the amount of sells in each weekday.

In [12]:
product_df_raw['DayOfWeek'] = product_df_raw['Reporting Day'].dt.day_name()

In [13]:
dow = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
       'Sunday']

sales_by_day = product_df_raw.groupby('DayOfWeek')['POS Total Units Sold Promo and Regular'].sum().reindex(dow)

In [14]:
sales_by_day.to_csv('/content/drive/MyDrive/Analytics/Dashboard/sales_day_df.csv', sep=';')

In [15]:
product_df_raw[['DayOfWeek',
                'POS Total Units Sold Promo and Regular', 'Encoded_REST_KEY']].to_csv('/content/drive/MyDrive/Analytics/Dashboard/sales_dist_df.csv', sep=';')

<h2> Demographics </h2>

In [16]:
demographics_df_raw = pd.read_csv('/content/drive/MyDrive/Analytics/Datasets/demographics_sample.csv')
demographics_df_raw

,Restaurant Key,Restaurant Open Date,Restaurant Close Date,Restaurant Coop,Restaurant Region,AACM,HCM,ACM,Household Count,Urban Uptown,...,Conservative Classics,Cautious Couples,Sustaining Seniors,Prom Soc,Prom Life,Row,Urban,Suburban,Second City,Town and Rural
0,REST_KEY,REST_OPN_DT,REST_CLS_DT,REST_MNT_ADV_COOP,REST_RGN,afr_amr_cons,hisp_cons,asian_cons,HOUSEHLDSC,Soc-U1,...,Life-M2,Life-M3,Life-M4,Prom Soc,Prom Life,Row,PNECY_URB,PNECY_SUB,PNECY_CITY,PNECY_TR
1,1364,2/26/70,NaN,SEA/TCA WA CP-0024,NORTHWEST REGION-0160480000,9.2,9.67,13.66,"9,080",0%,...,21%,1%,2%,Soc-C2,Life-Y2,"13,055",0%,0%,100%,0%
2,5357,2/6/80,NaN,SEA/TCA WA CP-0024,NORTHWEST REGION-0160480000,9.24,11.34,42.83,"1,281",10%,...,6%,17%,0%,Soc-S3,Life-Y1,"10,213",13%,87%,0%,0%
3,13369,11/3/95,NaN,SEA/TCA WA CP-0024,NORTHWEST REGION-0160480000,8.91,22.72,19.63,"3,108",0%,...,11%,16%,0%,Soc-S4,Life-F3,"5,748",0%,93%,7%,0%
4,13604,3/8/96,NaN,SEA/TCA WA CP-0024,NORTHWEST REGION-0160480000,3.2,6.28,6.89,"9,219",0%,...,13%,0%,0%,Soc-C1,Life-Y1,"5,652",0%,7%,93%,0%


In [17]:
#Reshaping our dataset by dropping row's 0:
demographics_df_raw.columns = demographics_df_raw.iloc[0,:].values
demographics_df_raw.drop(0, axis=0, inplace=True)

In [18]:
#Encoding the variable 'REST_KEY' for readability and replication:
# Define the mapping dictionary
mapping_dict_2 = {
    1364: 'Store1',
    5357: 'Store2',
    13369: 'Store3',
    13604: 'Store4'
}

# Convert REST_KEY to integer if necessary
demographics_df_raw['REST_KEY'] = demographics_df_raw['REST_KEY'].astype(int)

# Encoding the variable 'REST_KEY'
encoded_rest_key_2 = demographics_df_raw['REST_KEY'].map(mapping_dict_2)

# Check for unmatched values
unmatched_values_2 = demographics_df_raw[~demographics_df_raw['REST_KEY'].isin(mapping_dict_2.keys())]
if not unmatched_values_2.empty:
    print("Unmatched REST_KEY values:")
    print(unmatched_values_2['REST_KEY'].unique())

# Add the encoded column to the DataFrame
demographics_df_raw['Encoded_REST_KEY'] = encoded_rest_key_2

print("Encoded REST_KEY Column:")
print(encoded_rest_key_2)

Encoded REST_KEY Column:
1    Store1
2    Store2
3    Store3
4    Store4
Name: REST_KEY, dtype: object


We keep the demographic statistics of interest to display a table with the important information regarding population.

In [19]:
grouped_df_2 = demographics_df_raw[['Encoded_REST_KEY', 'REST_OPN_DT',
                                    'afr_amr_cons', 'hisp_cons',
                                    'asian_cons','HOUSEHLDSC', 'Soc-U1',
                                    'Soc-S2','Soc-S3', 'Life-F2', 'Life-M3',
                                    'PNECY_URB']].groupby('Encoded_REST_KEY').sum()

In [20]:
grouped_df_2.to_csv('/content/drive/MyDrive/Analytics/Dashboard/demographics_df.csv', sep=';')

In [21]:
grouped_df_2

,REST_OPN_DT,afr_amr_cons,hisp_cons,asian_cons,HOUSEHLDSC,Soc-U1,Soc-S2,Soc-S3,Life-F2,Life-M3,PNECY_URB
Encoded_REST_KEY,,,,,,,,,,,
Store1,2/26/70,9.2,9.67,13.66,"9,080",0%,0%,0%,1%,1%,0%
Store2,2/6/80,9.24,11.34,42.83,"1,281",10%,18%,51%,13%,17%,13%
Store3,11/3/95,8.91,22.72,19.63,"3,108",0%,8%,41%,3%,16%,0%
Store4,3/8/96,3.2,6.28,6.89,"9,219",0%,3%,1%,1%,0%,0%


<h2> Dayparts </h2>

In [22]:
dayparts_df_raw = pd.read_csv('/content/drive/MyDrive/Analytics/Datasets/dayparts_sample.csv')
dayparts_df_raw

,REST_KEY,Rest Coop,Reporting Day,Daypart Name,Daypart Description,Daypart Sales $,Daypart Transaction Qty
0,1364,SEA/TCA WA CP-0024,1/2/10,After 4PM,4PM - Close,1529.84,235
1,1364,SEA/TCA WA CP-0024,1/2/10,Afternoon,2PM - 4PM,539.20,75
2,1364,SEA/TCA WA CP-0024,1/2/10,Breakfast,Open - 10:30AM (11AM weekend),1027.40,180
3,1364,SEA/TCA WA CP-0024,1/2/10,Dinner,4PM - 9PM,957.24,156
4,1364,SEA/TCA WA CP-0024,1/2/10,Evening,9PM - Close,572.60,79
...,...,...,...,...,...,...,...
2445,13604,SEA/TCA WA CP-0024,1/30/11,Late Night,11PM - 5AM,0.00,0
2446,13604,SEA/TCA WA CP-0024,1/30/11,Lunch,11AM - 2PM,917.64,149
2447,13604,SEA/TCA WA CP-0024,1/30/11,Morning,Open - 11AM,907.15,160
2448,13604,SEA/TCA WA CP-0024,1/30/11,Regular Menu,10:30AM (11AM weekend) - Close,2119.59,327


In [23]:
#Encoding the variable 'REST_KEY' for readability and replication:
# Define the mapping dictionary
mapping_dict_3 = {
    1364: 'Store1',
    5357: 'Store2',
    13369: 'Store3',
    13604: 'Store4'
}

# Convert REST_KEY to integer if necessary
dayparts_df_raw['REST_KEY'] = dayparts_df_raw['REST_KEY'].astype(int)

# Encoding the variable 'REST_KEY'
encoded_rest_key_3 = dayparts_df_raw['REST_KEY'].map(mapping_dict_3)

# Check for unmatched values
unmatched_values_3 = dayparts_df_raw[dayparts_df_raw['REST_KEY'].isin(mapping_dict_3.keys())]
if not unmatched_values_3.empty:
    print("Unmatched REST_KEY values:")
    print(unmatched_values_3['REST_KEY'].unique())

# Add the encoded column to the DataFrame
dayparts_df_raw['Encoded_REST_KEY'] = encoded_rest_key_3

print("Encoded REST_KEY Column:")
print(encoded_rest_key_3)

Unmatched REST_KEY values:
[ 1364  5357 13369 13604]
Encoded REST_KEY Column:
0       Store1
1       Store1
2       Store1
3       Store1
4       Store1
         ...  
2445    Store4
2446    Store4
2447    Store4
2448    Store4
2449    Store4
Name: REST_KEY, Length: 2450, dtype: object


In [24]:
dayparts_df = dayparts_df_raw.drop(columns=['REST_KEY', 'Rest Coop' ])

In [25]:
dayparts_df['Reporting Day'] = pd.to_datetime(dayparts_df['Reporting Day'])

<ipython-input-25-e49275d8b636>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dayparts_df['Reporting Day'] = pd.to_datetime(dayparts_df['Reporting Day'])


In [26]:
# Filter for January 2011
january_2011_data = dayparts_df[
    (dayparts_df['Reporting Day'] >= '2011-01-01') &
    (dayparts_df['Reporting Day'] < '2011-02-01')
]

In [27]:
january_2011_data[['Daypart Name', 'Daypart Sales $', 'Encoded_REST_KEY']].to_csv('/content/drive/MyDrive/Analytics/Dashboard/daypart_df.csv', sep=';')

In [28]:
january_2011_data[['Daypart Name', 'Daypart Sales $', 'Encoded_REST_KEY']]

,Daypart Name,Daypart Sales $,Encoded_REST_KEY
310,After 4PM,1627.39,Store1
311,Afternoon,736.38,Store1
312,Breakfast,646.95,Store1
313,Dinner,1375.57,Store1
314,Evening,251.82,Store1
...,...,...,...
2445,Late Night,0.00,Store4
2446,Lunch,917.64,Store4
2447,Morning,907.15,Store4
2448,Regular Menu,2119.59,Store4


In [29]:
january_2011_data[january_2011_data['Encoded_REST_KEY']=='Store1']['Daypart Name'].value_counts()

Daypart Name
After 4PM       31
Afternoon       31
Breakfast       31
Dinner          31
Evening         31
Late Night      31
Lunch           31
Morning         31
Regular Menu    31
Total Day       31
Name: count, dtype: int64

In [30]:
data_filtered = january_2011_data.copy()

# Daily aggregation
daily_aggregation = data_filtered.groupby(['Encoded_REST_KEY','Reporting Day']).agg({
    'Daypart Sales $': 'sum',
    'Daypart Transaction Qty': 'sum'
}).reset_index()

In [31]:
daily_aggregation.to_csv('/content/drive/MyDrive/Analytics/Dashboard/transactions_df.csv', sep=';')

In [32]:
daily_aggregation

,Encoded_REST_KEY,Reporting Day,Daypart Sales $,Daypart Transaction Qty
0,Store1,2011-01-01,13471.93,2012
1,Store1,2011-01-02,13434.07,2129
2,Store1,2011-01-03,13164.88,2501
3,Store1,2011-01-04,13278.05,2470
4,Store1,2011-01-05,13485.78,2621
...,...,...,...,...
116,Store4,2011-01-26,11992.18,2432
117,Store4,2011-01-27,15143.48,2823
118,Store4,2011-01-28,14036.74,2531
119,Store4,2011-01-29,15470.57,2605
